In [ ]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 9.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("mlewand/PROTBERT")
model = AutoModelForMaskedLM.from_pretrained("mlewand/PROTBERT")

Some weights of the model checkpoint at mlewand/PROTBERT were not used when initializing BertForMaskedLM: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at mlewand/PROTBERT and are newly initialized: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
You should probably TRAIN this model on a down-stream task to be 

In [ ]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.mask_token_id)
print(tokenizer.pad_token_id)

0
2
4
1


# Imports

In [ ]:
import os
import requests
import tqdm
import json
import pandas as pd
import numpy as np
import zipfile
from collections import Counter

import torch

# Hyperparameters

In [ ]:
file_id = '1w3IQMK3PmXH-Bq6Lt_P8wxGPvr5olrZT'
destination = 'pfam.zip'
hf_name = "t5-small"
n_families_of_interest = 1
data_dirpath = "pfam"
aminoacid_separate_by = ""
max_length = 512
tokenizer_folder = "PROTNAME_tok"
save_folder = "PROTNAME"
vocab_size=32

# Utils

In [ ]:
def format_size(value):
  if value >= 1024**3:
    return f"{round(value / 1024**3 , 3)} GB"
  elif value >= 1024**2:
    return f"{round(value / 1024**2 , 3)} MB"
  elif value >= 1024:
    return f"{round(value / 1024 , 3)} KB"
  return f"{value} B"

In [ ]:
# Code taken from https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    params = { 'id' : id, 'confirm' : 1 }
    response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    pb = tqdm.tqdm(response.iter_content(CHUNK_SIZE))
    b_total = 0
    with open(destination, "wb") as f:
        for i,chunk in enumerate(pb):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                b_total += len(chunk)

                if i % 1000 == 0:
                  pb.set_description(f"written : {format_size(b_total)}")
                i += 1
    print("\n")
    print(f"saved all the data to {destination}. total size : {format_size(os.stat(destination).st_size)}")

# Get the Dataset

In [ ]:
download_file_from_google_drive(file_id, destination)
with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall(".")
!mv random_split pfam

written : 468.781 MB: : 15780it [00:03, 4197.20it/s]




saved all the data to pfam.zip. total size : 493.095 MB


# Read Data

In [ ]:
def read_all_shards(partition='dev', data_dir = data_dirpath):
    shards = []
    for fn in os.listdir(os.path.join(data_dir, partition)):
        with open(os.path.join(data_dir, partition, fn)) as f:
            shards.append(pd.read_csv(f, index_col=None))
    
    return pd.concat(shards)

def read_all_data_initial():
  global train, test, dev, all_train_ds_size, all_test_ds_size, all_dev_ds_size

  test = read_all_shards('test')
  dev = read_all_shards('dev')
  train = read_all_shards('train')

  partitions = {'test': test, 'dev': dev, 'train': train}
  for name, df in partitions.items():
      print('Dataset partition "%s" has %d sequences' % (name, len(df)))

  all_train_ds_size = len(train)
  all_test_ds_size = len(test)
  all_dev_ds_size = len(dev)

  train.reset_index(inplace=True, drop=True)
  dev.reset_index(inplace=True, drop=True)
  test.reset_index(inplace=True, drop=True)

def get_cumulative(data):
    counter = Counter(data['family_accession'])
    print(f"how many labels : {len(counter)}")
    
    datasetSize = len(data)
    xs = []
    x_labels = []
    ys = []

    t = 0
    cumulative = []

    for i,(x, y) in  enumerate(counter.most_common()):
        xs.append(i)
        x_labels.append(x)
        ys.append(y)
        t += y / datasetSize
        cumulative.append(t)
    return cumulative


# EXECUTION CODE
print('Available dataset partitions: ', os.listdir(data_dirpath))
read_all_data_initial()
cumulative = get_cumulative(train)
print(f"{n_families_of_interest} classes is {100 * round( cumulative[n_families_of_interest-1],3)} portion of training data")

familiesOfInterest = train.family_accession.value_counts()[:n_families_of_interest]

mask = train.family_accession.isin(familiesOfInterest.index.values)
train = train.loc[mask,:]

mask = dev.family_accession.isin(familiesOfInterest.index.values)
dev = dev.loc[mask,:]

mask = test.family_accession.isin(familiesOfInterest.index.values)
test = test.loc[mask,:]


################################################################################
train_seq = train['sequence']
dev_seq = dev['sequence']
test_seq = test['sequence']

################################################################################
train_sentences = train_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
validation_sentences = dev_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))
test_sentences = test_seq.apply(lambda seq: aminoacid_separate_by.join([aa for aa in seq]))

################################################################################
train_labels = train['family_accession'].apply(lambda x: x.split('.')[0])
validation_labels = dev['family_accession'].apply(lambda x: x.split('.')[0])
test_labels = test['family_accession'].apply(lambda x: x.split('.')[0])

Available dataset partitions:  ['train', 'test', 'random_split', 'dev']
Dataset partition "test" has 126171 sequences
Dataset partition "dev" has 126171 sequences
Dataset partition "train" has 1086741 sequences
how many labels : 17929
1 classes is 0.3 portion of training data


In [ ]:
train_sentences

450        IIDIGCGDGARTLANLPAESVGIDLSRTGLQLAADTVPSATLVQAD...
714        ILDLGCGTGSHLLHYLQQGAKQVIGIDLSTKMLEQATKNLKSAGFN...
745        ILSVGCGNSPMSAQLLKEGASKVYNVDFSHVVIDQMKALHQEESNL...
1178       ILDVGCGPGSITAGFAQYVPEGTVVGVDISAEVLARATQRAAELGL...
1336       VLDVGTGTGILGIASAKLGAERVLCVDIDPKATEIAKENIAINHVE...
                                 ...                        
1085697    ALDVCCGTADWTIALAEAVGPSGEVYGLDFSKNMLQVGEQKVKERQ...
1085732    VLDVGTGTGVLVPFLNEAGCRIDAVDISEKMLEVARAKFPGMANFI...
1085863    VLDVGCGSGAITADIAQKVGKNGRVIGIDYSEHLIELAKINYSSIE...
1085974    VLDVATGTGWTARSIARSGAIVSGVDLGVDLVTAAQNRARAEGLNI...
1086086    VLDAGCGVSHVAIHLAAKHGLKVQGIDIVDHHIVKARRNIARSGLS...
Name: sequence, Length: 3637, dtype: object

In [ ]:
labels = set()

for v in tqdm.tqdm(train_labels):
  labels.add(v)

label_to_id = {k:v for v,k in enumerate(labels)}

print()
print(len(labels))

100%|██████████| 3637/3637 [00:00<00:00, 500941.93it/s]


1


In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({"sentence": train_sentences})
dataset_val = Dataset.from_dict({"sentence": validation_sentences})

In [ ]:
dataset

Dataset({
    features: ['sentence'],
    num_rows: 3637
})

In [ ]:
datapoint = train_sentences.iloc[0]
encoded = tokenizer.encode(datapoint)
print(f"datapoint : {datapoint}, len({len(datapoint)})")
print(encoded)

datapoint : IIDIGCGDGARTLANLPAESVGIDLSRTGLQLAADTVPSATLVQADMASLPVAENTADAITASHAVFHVPRSDHPTVYREVARVLKPGG, len(89)
[0, 45, 45, 40, 45, 43, 39, 43, 40, 43, 37, 54, 56, 48, 37, 50, 48, 52, 37, 41, 55, 58, 43, 45, 40, 48, 55, 54, 56, 43, 48, 53, 48, 37, 37, 40, 56, 58, 52, 55, 37, 56, 48, 58, 53, 37, 40, 49, 37, 55, 48, 52, 58, 37, 41, 50, 56, 37, 40, 37, 45, 56, 37, 55, 44, 37, 58, 42, 44, 58, 52, 54, 55, 40, 44, 52, 56, 58, 61, 54, 41, 58, 37, 54, 58, 48, 47, 52, 43, 43, 2]


In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset_val = dataset_val.map(preprocess_function, batched=True)

Map:   0%|          | 0/3637 [00:00<?, ? examples/s]

Map:   0%|          | 0/454 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.05)
tok = tokenizer(train_sentences.iloc[0], truncation=True)
data_collator([tok])

{'input_ids': tensor([[ 0, 45, 45, 40, 45, 43, 39, 43, 40, 43, 37, 54, 56, 48, 37, 50, 48, 52,
         37, 41, 55, 58, 43, 45, 40, 48, 55, 54, 56, 43, 48, 53, 48, 37, 37, 40,
         56, 58, 52, 55, 37, 56, 48, 58, 53, 37, 40, 49, 37, 55, 48, 52, 58, 37,
         41, 50, 56, 37, 40, 37, 45, 56, 37, 55, 44, 37, 58, 42, 44, 58, 52, 54,
         55, 40, 44, 52, 56, 58, 61, 54, 41, 58, 37,  4, 58, 48, 47, 52, 43,  4,
          2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  # labels = np.argmax(labels, axis=1)
  preds = pred.predictions.argmax(-1)

  tot = np.sum(labels != -100) + 1e-10
  classified_correctly = 0
  for i in range(preds.shape[0]):
    mask = preds[i, :] != -100
    classified_correctly += np.sum(preds[i, mask] == labels[i, mask])
    

  print(classified_correctly, tot)
  #print(f'predictions: {labels.shape}')
  #print(f'labels: {labels.shape}')
  # print(labels[0])
  # print(preds[0])
  #cnt = 0
  # for i in range(len(labels)):
  #   if labels[i] == preds[i]:
  #     cnt += 1
  
  return {"exact_matches": classified_correctly / tot}

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir=save_folder,
    overwrite_output_dir=True,
    learning_rate=5e-4,
    num_train_epochs=10,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    evaluation_strategy="epoch",
    logging_dir=os.path.join(save_folder, "logs"),            # directory for storing logs
    logging_steps=50,
    save_total_limit=2,
    prediction_loss_only=False,
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator = data_collator
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Exact Matches
1,2.090700,2.132875,0.348805
2,2.090700,2.139904,0.361582
3,2.109200,2.085364,0.379838
4,2.105200,2.087261,0.383217
5,2.119600,2.106383,0.365403
6,2.092900,2.109606,0.359247
7,2.085100,2.054416,0.381519
8,2.096000,2.036507,0.391462
9,2.096800,2.075217,0.355269
10,2.091700,2.052304,0.378781


744 2133.0000000001
768 2124.0000000001
795 2093.0000000001
822 2145.0000000001
771 2110.0000000001
744 2071.0000000001
834 2186.0000000001
862 2202.0000000001
772 2173.0000000001
814 2149.0000000001


TrainOutput(global_step=290, training_loss=2.097779662033607, metrics={'train_runtime': 30.5917, 'train_samples_per_second': 1188.886, 'train_steps_per_second': 9.48, 'total_flos': 41431273539024.0, 'train_loss': 2.097779662033607, 'epoch': 10.0})

In [ ]:
encoded_dataset_val

In [ ]:
from  transformers import pipeline

model=model.to("cpu")
p = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
seq = train_sentences.iloc[0]
seq = seq[:-1]
seq += '<mask>'

p(seq)

In [ ]:
!pip install torchinfo
from torchinfo import summary 

In [ ]:
summary(model)